In [38]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score)

In [39]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
#Importing the datasets
# Import the training dataset with only the important colums
train_df = pd.read_csv("datasets/train.csv")

# Column 3 is the column of interest in the testing set
test_df = pd.read_csv("datasets/test.csv")



In [41]:
#Explore train dataset
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [42]:
test_df.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [43]:
# Cleaning the dataset

def clean_text(text):
    if isinstance(text, str):
        # Remove non-alphanumeric characters and convert to lowercase
        text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

        # Tokenize the text into individual words
        words = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]

        # Join the words back into a single string
        clean_text = ' '.join(words)

        return clean_text


In [44]:
#Applying preprocessing function on the text column
train_df['cleaned_text'] = train_df['text'].apply(clean_text)

# Converting sentiments to numerical form
sentiment_mapping = {'neutral': 0, 'positive': 1, 'negative': -1}
train_df['sentiment'] = train_df['sentiment'].map(sentiment_mapping)

train_df.dropna()

# Print the preprocessed train dataset
print(train_df.head())

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text  sentiment  \
0  I`d have responded, if I were going          0   
1                             Sooo SAD         -1   
2                          bullying me         -1   
3                       leave me alone         -1   
4                        Sons of ****,         -1   

                       cleaned_text  
0                   responded going  
1           sooo sad miss san diego  
2                     boss bullying  
3             interview leave alone  
4  sons put releases already bought  


In [45]:
# Filter out rows with None values in the 'cleaned_text' column
train_df_filtered = train_df.dropna(subset=['cleaned_text'])

#Initializing and fitting the vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df_filtered['cleaned_text'])

# Transforming the preprocessed text data into TF-IDF features
features = vectorizer.transform(train_df_filtered['cleaned_text'])

# Convert the features into a DataFrame
features_df = pd.DataFrame(features.toarray(), columns=vectorizer.get_feature_names_out())

train_features_df = pd.concat([train_df_filtered, features_df], axis=1)

# Print the train dataset with TF-IDF features
print(train_features_df.head())


       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text  sentiment  \
0  I`d have responded, if I were going        0.0   
1                             Sooo SAD       -1.0   
2                          bullying me       -1.0   
3                       leave me alone       -1.0   
4                        Sons of ****,       -1.0   

                       cleaned_text   00  000  000th  007  00am  ...  zune  \
0                   responded going  0.0  0.0    0.0  0.0   0.0  ...   0.0   
1           sooo sad miss san diego  0.0  0.0    0.0  0.0   0.0  ...   0.0   
2                     boss bullying  0